# Memoria en Agent Framework (Threads vs ChatMessageStore)

Este notebook se centra en **cómo gestionar memoria y persistencia**.

## Objetivo
Vamos a ver 2 formas principales de gestionar memoria:

1) **Threads (AgentThread)** para conversaciones multi-turn (estado conversacional).
2) **ChatMessageStore** para controlar *dónde* se guardan los mensajes, y cómo **persistirlos a JSON**.

Y después veremos opciones para persistir/recordar en remoto (Redis, Mem0, etc.).

## Conceptos

- Un **AgentThread** mantiene el estado y el historial de conversación para un agente.
- El historial puede estar:
  - **en el propio thread** (caso típico cuando el servicio subyacente no guarda historial), o
  - **en un servicio remoto**, y el thread solo mantiene un **ID de referencia**.
- En el modo **local**, el thread usa un **ChatMessageStore** para almacenar y recuperar mensajes.

Idea práctica:
- **Thread** = *la conversación (estado) / tu identidad como usuario*
- **ChatMessageStore** = *el almacén de mensajes (dónde viven: RAM, JSON, Redis, DB...)*

### ¿Por qué reutilizar THREAD da memoria, pero STORE solo no?

**Thread SIN ChatMessageStore explícito → ✅ Recuerda**
- El thread es un **objeto con historial embebido en RAM**.
- Si pasas el mismo `thread` en cada turno, ese objeto mantiene todo el historial.
- El agente dice: "eres tú de nuevo → aquí está tu historial guardado".

**ChatMessageStore SOLO (sin thread reutilizado) → ❌ NO recuerda**
- El store es un contenedor **pasivo** (no tiene identidad).
- Si no pasas `thread`, cada `agent.run()` crea un **thread temporal nuevo**.
- Ese thread nuevo crea su propio store nuevo/vacío (tu store anterior queda huérfano).
- Es como tener un archivo perfecto pero perder tu carné de identidad: nadie sabe quién eres.

**La regla:** `thread` = identidad. Sin identidad, no hay "continuidad", aunque tengas el almacén.

In [1]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv

from agent_framework.openai import OpenAIChatClient
from agent_framework import ChatMessageStore

In [2]:
# Cargar credenciales (.env)
load_dotenv()

base_url = os.getenv('AZURE_OPENAI_ENDPOINT')
api_key = os.getenv('AZURE_OPENAI_API_KEY')
model_id = os.getenv('AZURE_OPENAI_DEPLOYMENT')

assert base_url, 'Falta AZURE_OPENAI_ENDPOINT en tu .env'
assert api_key, 'Falta AZURE_OPENAI_API_KEY en tu .env'
assert model_id, 'Falta AZURE_OPENAI_DEPLOYMENT en tu .env'

client = OpenAIChatClient(base_url=base_url, api_key=api_key, model_id=model_id)
print('✅ OpenAIChatClient listo:', model_id)

✅ OpenAIChatClient listo: gpt-4


In [4]:
# Agente base (sin store explícito).
# En este modo, el framework gestiona el historial de forma 'local' (en proceso) si aplica.
agent_threads = client.create_agent(
    name='SistemaAsistenciaMemoria_Threads',
    instructions=(
        'Eres un asistente médico. Mantén el contexto de la conversación y responde de forma clara.'
    ),
)
print('✅ Agent creado: Threads (modo básico)')

✅ Agent creado: Threads (modo básico)


## 1) Forma 1: Usar Threads para multi-turn (memoria de conversación)

Regla de oro: **si quieres que recuerde, reutiliza el mismo `thread`**.

En el ejemplo, hacemos dos preguntas en el mismo thread y después repetimos sin pasar thread para ver la diferencia.

In [5]:
# Ejemplo extendido: multi-turn realista con el MISMO thread
# - Un thread por "sesión" (o por paciente / caso)
# - Reutilizar el mismo thread mantiene contexto
# - Un thread distinto NO comparte memoria conversacional

async def run_turns(agent, thread, turns, title=None):
    if title:
        print(f"\n=== {title} ===")
    for i, user_msg in enumerate(turns, start=1):
        res = await agent.run(user_msg, thread=thread)
        print(f"U{i}: {user_msg}")
        print(f"A{i}: {res.text}\n")

# Thread 1: "Caso A" (misma conversación)
thread_a = agent_threads.get_new_thread()

turns_a = [
    "Mi nombre es William Riker. Estoy en la USS Enterprise.",
    "Estoy mareado desde esta mañana y tengo dolor de cabeza.",
    "No tengo fiebre, pero dormí poco y hoy no he bebido agua.",
    "Con todo lo anterior, ¿qué crees que podría tener y qué pasos iniciales recomiendas?",
    "Recuérdame: ¿cómo me llamo y dónde estoy?",
]

await run_turns(agent_threads, thread_a, turns_a, title="Caso A (mismo thread, mantiene contexto)")



=== Caso A (mismo thread, mantiene contexto) ===
U1: Mi nombre es William Riker. Estoy en la USS Enterprise.
A1: ¡Encantado de conocerlo, Comandante Riker! ¿En qué puedo ayudarle hoy? Si tiene alguna consulta médica, síntomas o dudas sobre su salud (ya sea en el USS Enterprise o en la Tierra), estoy aquí para asistirlo.

U2: Estoy mareado desde esta mañana y tengo dolor de cabeza.
A2: Gracias por compartir cómo se siente, Comandante Riker. El mareo y el dolor de cabeza pueden ser causados por varias razones, desde causas leves como fatiga, deshidratación o estrés, hasta otras condiciones que pueden requerir atención.

¿Podría aclararme algunas cosas para orientarle mejor?

1. ¿Tiene otros síntomas acompañantes? (fiebre, náuseas, vómitos, visión borrosa, debilidad, dificultad para hablar, dolor en el pecho, o zumbidos en los oídos).
2. ¿Es la primera vez que le ocurre o ya le ha pasado antes?
3. ¿Ha cambiado algo en su rutina, dieta o medicación últimamente?
4. ¿Ha tenido lesiones en l

In [6]:
# Thread 2: "Caso B" (conversación independiente)
# Nota: aquí preguntamos lo mismo sin haber dado el contexto, para ver el contraste.
thread_b = agent_threads.get_new_thread()

turns_b = [
    "Recuérdame: ¿cómo me llamo y dónde estoy?",
    "¿Qué información te faltaría para orientarme mejor?",
]

await run_turns(agent_threads, thread_b, turns_b, title="Caso B (otro thread, NO comparte contexto)")

# Contraste adicional: llamada SIN thread (thread temporal). Normalmente equivale a "sin memoria".
res_temp = await agent_threads.run("Recuérdame: ¿cómo me llamo y dónde estoy?")
print("\n=== Llamada SIN thread (temporal) ===")
print(res_temp.text)


=== Caso B (otro thread, NO comparte contexto) ===
U1: Recuérdame: ¿cómo me llamo y dónde estoy?
A1: Lo siento, pero no tengo información almacenada sobre tu nombre ni tu ubicación, ya que no tengo acceso a datos personales previos ni registro de tus mensajes anteriores. Si quieres, puedes decirme tu nombre y dónde estás, y podré tenerlo en cuenta en esta conversación. ¿En qué puedo ayudarte hoy?

U2: ¿Qué información te faltaría para orientarme mejor?
A2: Gracias por tu interés en recibir una mejor orientación. Para poder ayudarte de manera más precisa y personalizada, me sería útil saber:

1. **Tu nombre o cómo prefieres que te llame.**
2. **Tu edad** (aproximada), ya que algunas indicaciones pueden variar según la etapa de la vida.
3. **Tu ubicación geográfica** (país o ciudad), porque algunas recomendaciones dependen de la disponibilidad de servicios médicos o normativas locales.
4. **Tu motivo de consulta o síntomas** si buscas ayuda médica.
5. **Antecedentes de salud relevantes*

### Resumen de "Forma 1"

| Aspecto | Resultado |
|--------|-----------|
| **Reutilizar mismo thread** | ✅ Recuerda todo (historial en RAM) |
| **Crear thread nuevo** | ❌ No recuerda (nuevo historial vacío) |
| **Sin pasar thread** | ❌ No recuerda (cada llamada = thread temporal) |
| **¿Persiste tras reiniciar?** | ❌ No (se pierde en RAM) |
| **Ideal para:** | Conversaciones cortas en la misma sesión |

**Conclusión:** Thread embebido = memoria fácil pero volátil.

## 2) Forma 2: ChatMessageStore + persistencia a JSON

Cuando el historial se gestiona localmente, el thread puede apoyarse en un **ChatMessageStore**.

### ¿Cuándo necesitas ChatMessageStore?

**Forma 1 (Thread solo) funciona bien MIENTRAS el objeto thread viva** (en RAM).  
Pero si cierras la app → pierdes todo.

**Forma 2 (Thread + ChatMessageStore)** permite:
- **Controlar dónde viven los mensajes** (RAM, JSON file, Redis, DB...).
- **Persistir y reanudar** una conversación: guardas el thread a JSON y después lo cargas.

### Cómo configurarlo

1) Configuras `chat_message_store_factory` en el agente: "así es cómo quiero que guardes mensajes".
2) Aún así, **debes reutilizar y pasar el `thread`** en cada turno (igual que Forma 1).
3) El thread usará ese tipo de store internamente.
4) Cuando termines, serializas el thread completo → se guarda con su historial → lo cargas después.

En el siguiente ejemplo:
- Creamos un agente con `chat_message_store_factory=ChatMessageStore` (in-memory pero serializable).
- Ejecutamos una conversación.
- Serializamos el **thread** (que incluye su store adentro) y lo guardamos a JSON.
- Recargamos el JSON y seguimos conversando (simula reinicio de la app).

In [7]:
def create_message_store():
    return ChatMessageStore()

agent_store = client.create_agent(
    name='SistemaAsistenciaMemoria_Store',
    instructions=(
        'Eres un asistente médico. Recuerda lo que el usuario dice y usa ese contexto después.'
    ),
    chat_message_store_factory=create_message_store,
)

thread_store = agent_store.get_new_thread()

res_a = await agent_store.run('Me llamo Deanna Troi y soy consejera.', thread=thread_store)
print('Turno A ->', res_a.text)

res_b = await agent_store.run('¿Cómo me llamo y cuál es mi rol?', thread=thread_store)
print('Turno B ->', res_b.text)

# Persistir thread a JSON
data_dir = Path('data')
data_dir.mkdir(exist_ok=True)
thread_json_path = data_dir / 'thread_state.json'

serialized_thread = await thread_store.serialize()
thread_json_path.write_text(json.dumps(serialized_thread, ensure_ascii=False, indent=2), encoding='utf-8')
print('✅ Thread serializado en:', thread_json_path.resolve())

Turno A -> Mucho gusto, Deanna Troi. Gracias por compartir tu nombre y tu profesión de consejera. ¿En qué puedo ayudarte hoy? Si tienes alguna consulta médica o necesitas información específica, estaré encantado de asistirte.
Turno B -> Tu nombre es Deanna Troi y tu rol es consejera. ¿En qué puedo ayudarte hoy?
✅ Thread serializado en: C:\Users\rnava\OneDrive - NTT DATA EMEAL\Trabajo\02.Evangelist\eventos\1_presentadas\AI Bootcamp 2026 AgentCamp(MAF)\codigo\data\thread_state.json


In [8]:
# Simular “reinicio”: cargamos JSON y reconstruimos el thread con el MISMO tipo de agente
loaded = json.loads(Path('data/thread_state.json').read_text(encoding='utf-8'))
resumed_thread = await agent_store.deserialize_thread(loaded)

res_c = await agent_store.run('Perfecto. Recuérdame otra vez mi nombre.', thread=resumed_thread)
print('Turno C (thread reanudado) ->', res_c.text)

Turno C (thread reanudado) -> Tu nombre es Deanna Troi. Si necesitas que recuerde algo más o si tienes alguna pregunta, aquí estoy para ayudarte.


### Resumen de "Forma 2" vs "Forma 1"

| Aspecto | Forma 1 (Thread solo) | Forma 2 (Thread + Store) |
|--------|--------|--------|
| **Necesito reutilizar thread?** | ✅ Sí | ✅ Sí (igual) |
| **¿Recuerda en la sesión?** | ✅ Sí | ✅ Sí |
| **¿Persiste tras cerrar?** | ❌ No | ✅ Sí (si guardas a JSON) |
| **¿Controlo dónde va?** | ❌ Solo RAM | ✅ Puedo elegir (JSON, Redis, DB) |
| **Ideal para:** | Pruebas / sesiones cortas | Producción / persistencia requerida |

**Punto clave:** La diferencia NO es "thread SÍ vs thread NO".  
La diferencia es **"thread en RAM" vs "thread serializado a archivo/BD"**.  
Ambas requieren **reutilizar el mismo thread para memoria**.

### (Extra) Serializar el ChatMessageStore directamente a JSON

A veces también interesa serializar/deserializar el **store** (no solo el thread).

Nota: en un escenario real, normalmente persistes el **thread** completo (porque incluye store + otros estados adjuntos).
Aquí lo hacemos como demo aislada de `ChatMessageStore.serialize()` / `ChatMessageStore.deserialize()`.

In [9]:
from agent_framework import ChatMessage

store = ChatMessageStore()
await store.add_messages([
    ChatMessage(role='user', content='Hola'), # type: ignore
    ChatMessage(role='assistant', content='Hola, ¿en qué puedo ayudarte?'), # type: ignore
])

store_state = await store.serialize()
Path('data/store_state.json').write_text(json.dumps(store_state, ensure_ascii=False, indent=2), encoding='utf-8')
print('✅ Store serializado en data/store_state.json')

restored_state = json.loads(Path('data/store_state.json').read_text(encoding='utf-8'))
restored_store = await ChatMessageStore.deserialize(restored_state)
messages = await restored_store.list_messages()
print('Mensajes restaurados:', len(messages))

✅ Store serializado en data/store_state.json
Mensajes restaurados: 2


## Persistencia / memoria remota (Redis, Mem0, etc.)

Una vez controlas `Thread` + `ChatMessageStore`, puedes llevarlo a producción con almacenes remotos.

### A) Redis como ChatMessageStore (historial persistente)
Esto mantiene el historial de chat en Redis (sobrevive reinicios) y sigue siendo *chat history*.

Ejemplo (requiere Redis disponible y el paquete correspondiente):
```python
from agent_framework.redis import RedisChatMessageStore

def create_redis_store():
    return RedisChatMessageStore(redis_url='redis://localhost:6379', thread_id='user_session_123', max_messages=200)

agent = client.create_agent(..., chat_message_store_factory=create_redis_store)
thread = agent.get_new_thread()
await agent.run('Hola', thread=thread)
```

### B) Mem0 como Context Provider (memoria semántica / largo plazo)
Mem0 **no** es un almacén del historial de chat; es un proveedor que extrae/inyecta recuerdos útiles.

Ejemplo:
```python
from agent_framework.mem0 import Mem0Provider
memory_provider = Mem0Provider(api_key='...', user_id='user_123', application_id='my_app')
agent = client.create_agent(..., context_providers=memory_provider)
```

### C) Otros backends
- Implementar tu propio `ChatMessageStoreProtocol` para DBs (CosmosDB, Postgres, etc.).
- Añadir otros `context_providers` (Redis provider, Azure AI Search provider, etc.) según integraciones.

## Resumen final

- **Threads** te dan multi-turn: si reutilizas el mismo `thread`, el agente mantiene el contexto.
- **ChatMessageStore** controla dónde viven los mensajes cuando el thread es local.
- Para **persistir a JSON**, lo más robusto es: `serialized = await thread.serialize()` y luego `await agent.deserialize_thread(serialized)`.
- Para producción: **Redis** (historial) y **Mem0** (memoria semántica/largo plazo) son patrones complementarios, no lo mismo.